## LSTM是一种特殊类型的循环神经网络

### LSTM的特点与结构

LSTM引入了一系列的“门”结构，这些门通过不同的机制控制着信息的流动和记忆的保持。这些门使得LSTM能够更好地捕捉长期依赖关系，适用于需要记忆长序列历史信息的任务。
LSTM从被设计之初就被用于解决一般递归神经网络中普遍存在的长期依赖问题，使用LSTM可以有效的传递和表达长时间序列中的信息并且不会导致长时间前的有用信息被忽略（遗忘）。与此同时，LSTM还可以解决RNN中的梯度消失/爆炸问题。

LSTM的核心组成部分如下：

1. **细胞状态（Cell State）**：这是LSTM的核心，用于在不同的时间步之间传递信息。细胞状态可以看作是一个长期记忆单元，能够保持长时间的信息。它允许网络在处理时间序列数据时能够记住之前的重要信息。
![RNN Cell](../img/4.png)
2. **输入门（Input Gate）**：输入门决定了哪些新的信息应该被添加到细胞状态中。它根据当前输入和先前的隐藏状态来控制信息的更新。输入门可以让网络选择性地记住一些输入信息。
C_{t-1} 更新为 C_{t}。前面的步骤已经决定了将会执行哪些操作，现在我们正式进行这些操作。
我们将旧的细胞状态 **C_{t-1}** 与 **f_{t}** 相乘，这样我们就丢弃了我们决定要丢弃的信息。
然后，我们将得到的结果与 **i_{t} * \hat{C}_{t}** 相加，其中 **i_{t}** 是输入门的输出，**\hat{C}_{t}** 是新的候选值。这一步完成后，我们就得到了新的细胞状态 **C_{t}**，它根据我们的决策对每个状态进行了相应的更新。
![RNN Cell](../img/5.png)
3. **遗忘门（Forget Gate）**：遗忘门负责从细胞状态中删除不再需要的旧信息。它决定了之前的记忆哪些值应该被遗忘，以便为新信息腾出空间。遗忘门帮助LSTM保持最新和最相关的信息。
在LSTM中的第一步是决定我们将从细胞状态中丢弃哪些信息。这个决策是由一个称为忘记门层（Forget Gate Layer）完成的。
该门层会读取上一时间步的隐藏状态 **h_{t-1}** 和当前时间步的输入 **x_{t}**，然后输出一个介于0到1之间的值，该值针对细胞状态 **C_{t-1}** 中的每个元素进行加权。
其中，1 表示“完全保留”，0 表示“完全舍弃”。
![RNN Cell](../img/6.png)
4. **输出门（Output Gate）**：输出门控制着从细胞状态到隐藏状态的信息流动。输出门基于当前输入和细胞状态，决定了隐藏状态中应该包含多少信息。这个隐藏状态随后会被用于生成预测结果。
最终，我们需要确定要输出的值。这个输出将基于我们的细胞状态，但经过了一次过滤。首先，我们使用一个 sigmoid 层来决定细胞状态的哪些部分将被输出。接下来，我们将细胞状态通过 tanh 函数进行处理，得到一个取值范围在 -1 到 1 之间的值。然后，我们将这个处理后的值与 sigmoid 门的输出相乘，这样，我们最终只会输出我们决定要输出的那部分内容。
![RNN Cell](../img/7.png)
### LSTM的应用领域

LSTM网络在许多领域中都表现出色，特别是那些涉及到序列数据和时间依赖性的任务。以下是一些LSTM在应用中的例子：

- **自然语言处理（NLP）**：LSTM在文本生成、情感分析、机器翻译等NLP任务中发挥着重要作用。其能够捕捉语言中的长期依赖关系，使得生成连贯的文本变得更加可靠。

- **语音识别**：LSTM在语音识别中能够处理变长的声学特征序列，从而实现更准确的语音识别。

- **图像描述生成**：将LSTM与卷积神经网络（CNN）结合，可以生成能够描述图像内容的自然语言文本。

- **股票预测**：LSTM可用于预测股票价格和市场走势，因为它能够捕捉历史数据中的时间相关性。

- **生物信息学**：LSTM在分析 DNA 序列、蛋白质结构预测等领域有潜在的应用，因为这些任务通常涉及到序列数据。

## 基于LSTM实现文本的情感分析
本文使用的亚马逊评论数据集，训练一个可以判别文本情感的分类器。

数据集如下：

链接：https://pan.baidu.com/s/1cK-scxLIliTsOPF-6byucQ
提取码：yqbq

In [ ]:
import bz2  # 用于读取bz2压缩文件
from collections import Counter  # 用于统计词频
import re  # 正则表达式
import nltk  # 文本预处理
import numpy as np

train_file = bz2.BZ2File('../data/amazon_reviews/train.ft.txt.bz2')
test_file = bz2.BZ2File('../data/amazon_reviews/test.ft.txt.bz2')
train_file = train_file.readlines()
test_file = test_file.readlines()
print(train_file[0])

### __label__1 代表差评，之后将其编码为0

### __label__2 代表好评，之后将其编码为1

In [ ]:
num_train = 800000
num_test = 200000

train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]

In [ ]:
# 将__label__1编码为0（差评），__label__2编码为1（好评）
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]
#对数据进行简单的数据清理

In [ ]:
#由于数字对情感分类帮助不大，所以这里将所有的数字都转换为0：
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d', '0', train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d', '0', test_sentences[i])

In [ ]:
#数据集中还存在包含网站的样本，例如：Welcome to our website: www.pohabo.com。
# 对于这种带有网站的样本，网站地址会干扰数据处理，所以一律处理成：Welcome to our website: <url>：
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])

for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [ ]:
#数据清理结束后，我们需要将文本进行分词，并将仅出现一次的单词丢掉，因为它们参考价值不大：
words = Counter()  # 用于统计每个单词出现的次数
for i, sentence in enumerate(train_sentences):
    words_list = nltk.word_tokenize(sentence)  # 将句子进行分词
    words.update(words_list)  # 更新词频列表
    train_sentences[i] = words_list  # 分词后的单词列表存在该列表中

    if i % 200000 == 0:  # 每20w打印一次进度
        print(str((i * 100) / num_train) + "% done")
print("100% done")

In [ ]:
words = {k: v for k, v in words.items() if v > 1}
#移除仅出现一次的单词

In [ ]:
words = sorted(words, key=words.get, reverse=True)
print(words[:10])  # 打印一下出现次数最多的10个单词
words = ['_PAD'] + words  #_PAD：表示填充，因为后续会固定所有句子长度。过长的句子进行阶段，过短的句子使用该单词进行填充
word2idx = {o: i for i, o in enumerate(words)}
idx2word = {i: o for i, o in enumerate(words)}
#映射字典准备完毕后，就可以将train_sentences中存储的单词转化为数字了
for i, sentence in enumerate(train_sentences):
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in
                         nltk.word_tokenize(sentence)]

In [ ]:
def pad_input(sentences, seq_len):
    """
    将句子长度固定为`seq_len`，超出长度的从后面阶段，长度不足的在前面补0
    """
    features = np.zeros((len(sentences), seq_len), dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features


# 固定测试数据集和训练数据集的句子长度
train_sentences = pad_input(train_sentences, 200)
test_sentences = pad_input(test_sentences, 200)

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

batch_size = 200

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size):
        super(SentimentNet, self).__init__()
        self.n_layers = n_layers = 2  # LSTM的层数
        self.hidden_dim = hidden_dim = 512  # 隐状态的维度，即LSTM输出的隐状态的维度为512
        embedding_dim = 400  # 将单词编码成400维的向量
        drop_prob = 0.5  # dropout

        # 定义embedding，负责将数字编码成向量，详情可参考：https://blog.csdn.net/zhaohongfei_358/article/details/122809709
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim,  # 输入的维度
                            hidden_dim,  # LSTM输出的hidden_state的维度
                            n_layers,  # LSTM的层数
                            dropout=drop_prob,
                            batch_first=True  # 第一个维度是否是batch_size
                            )

        # LSTM结束后的全连接线性层
        self.fc = nn.Linear(in_features=hidden_dim,  # 将LSTM的输出作为线性层的输入
                            out_features=1  # 由于情感分析只需要输出0或1，所以输出的维度是1
                            )
        self.sigmoid = nn.Sigmoid()  # 线性层输出后，还需要过一下sigmoid

        # 给最后的全连接层加一个Dropout
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x, hidden):
        """
        x: 本次的输入，其size为(batch_size, 200)，200为句子长度
        hidden: 上一时刻的Hidden State和Cell State。类型为tuple: (h, c),
        其中h和c的size都为(n_layers, batch_size, hidden_dim), 即(2, 200, 512)
        """
        # 因为一次输入一组数据，所以第一个维度是batch的大小
        batch_size = x.size(0)

        # 由于embedding只接受LongTensor类型，所以将x转换为LongTensor类型
        x = x.long()

        # 对x进行编码，这里会将x的size由(batch_size, 200)转化为(batch_size, 200, embedding_dim)
        embeds = self.embedding(x)

        # 将编码后的向量和上一时刻的hidden_state传给LSTM，并获取本次的输出和隐状态（hidden_state, cell_state）
        # lstm_out的size为 (batch_size, 200, 512)，200是单词的数量，由于是一个单词一个单词送给LSTM的，所以会产生与单词数量相同的输出
        # hidden为tuple(hidden_state, cell_state)，它们俩的size都为(2, batch_size, 512), 2是由于lstm有两层。由于是所有单词都是共享隐状态的，所以并不会出现上面的那个200
        lstm_out, hidden = self.lstm(embeds, hidden)

        # 接下来要过全连接层，所以size变为(batch_size * 200, hidden_dim)，
        # 之所以是batch_size * 200=40000，是因为每个单词的输出都要经过全连接层。
        # 换句话说，全连接层的batch_size为40000
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # 给全连接层加个Dropout
        out = self.dropout(lstm_out)

        # 将dropout后的数据送给全连接层
        # 全连接层输出的size为(40000, 1)
        out = self.fc(out)

        # 过一下sigmoid
        out = self.sigmoid(out)

        # 将最终的输出数据维度变为 (batch_size, 200)，即每个单词都对应一个输出
        out = out.view(batch_size, -1)

        # 只去最后一个单词的输出
        # 所以out的size会变为(200, 1)
        out = out[:, -1]

        # 将输出和本次的(h, c)返回
        return out, hidden

    def init_hidden(self, batch_size):
        """
        初始化隐状态：第一次送给LSTM时，没有隐状态，所以要初始化一个
        这里的初始化策略是全部赋0。
        这里之所以是tuple，是因为LSTM需要接受两个隐状态hidden state和cell state
        """
        hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device),
                  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
                  )
        return hidden

In [ ]:
model = SentimentNet(len(words))
model.to(device)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
lr = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
epochs = 2  # 一共训练两轮
counter = 0  # 用于记录训练次数
print_every = 1000  # 每1000次打印一下当前状态

for i in range(epochs):
    h = model.init_hidden(batch_size)  # 初始化第一个Hidden_state

    for inputs, labels in train_loader:  # 从train_loader中获取一组inputs和labels
        counter += 1  # 训练次数+1

        # 将上次输出的hidden_state转为tuple格式
        # 因为有两次，所以len(h)==2
        h = tuple([e.data for e in h])

        # 将数据迁移到GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # 清空模型梯度
        model.zero_grad()

        # 将本轮的输入和hidden_state送给模型，进行前向传播，
        # 然后获取本次的输出和新的hidden_state
        output, h = model(inputs, h)

        # 将预测值和真实值送给损失函数计算损失
        loss = criterion(output, labels.float())

        # 进行反向传播
        loss.backward()

        # 对模型进行裁剪，防止模型梯度爆炸
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

        # 更新权重
        optimizer.step()

        # 隔一定次数打印一下当前状态
        if counter % print_every == 0:
            print("Epoch: {}/{}...".format(i + 1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()))

In [ ]:
test_losses = []  # 记录测试数据集的损失
num_correct = 0  # 记录正确预测的数量
h = model.init_hidden(batch_size)  # 初始化hidden_state和cell_state
model.eval()  # 将模型调整为评估模式

# 开始评估模型
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # 将模型四舍五入为0和1
    correct_tensor = pred.eq(labels.float().view_as(pred))  # 计算预测正确的数据
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct / len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc * 100))

In [ ]:
def predict(sentence):
    # 将句子分词后，转换为数字
    sentences = [[word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]]

    # 将句子变为固定长度200
    sentences = pad_input(sentences, 200)

    # 将数据移到GPU中
    sentences = torch.Tensor(sentences).long().to(device)

    # 初始化隐状态
    h = (torch.Tensor(2, 1, 512).zero_().to(device),
         torch.Tensor(2, 1, 512).zero_().to(device))
    h = tuple([each.data for each in h])

    # 预测
    if model(sentences, h)[0] >= 0.5:
        print("positive")
    else:
        print("negative")

In [ ]:
#预测
predict("The film is so boring")
predict("The actor is too ugly.")